In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = 'validation_results_llama_4X.csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure
df.head()

,,,,,,,Solidity_file_name;vulnerability_type;vulnerable_line_number;vulnerability_function_entry_line;vulnerability_function_end_line;VL;pre_post_;pre;post;pre_VL_post;pre_VL;VL_post
dos_address.sol;denial_of_service;15;14;20;{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': False}
dos_number.sol;denial_of_service;17;14;24;{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True}
dos_simple.sol;denial_of_service;16;14;26;{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': True}
auction.sol;denial_of_service;22;15;28;{'Sanity_Test_Success': False,'Exploit_Covered': False};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True};{'Sanity_Test_Success': False,'Exploit_Covered': True}
name_registrar.sol;other;22;20;31;{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': True};{'Sanity_Test_Success': True,'Exploit_Covered': False};{'Sanity_Test_Success': True,'Exploit_Covered': True}


In [2]:
import ast

# Reload the file using the correct delimiter
df = pd.read_csv(file_path, delimiter=';')

# Define the result columns we are interested in
result_columns = ['VL', 'pre_post_', 'pre', 'post', 'pre_VL_post', 'pre_VL', 'VL_post']

# Function to safely convert string to dictionary
def parse_result_column(value):
    try:
        return ast.literal_eval(value)
    except Exception:
        return {}

# Apply parsing to each result column
for col in result_columns:
    df[col] = df[col].apply(parse_result_column)

# Extract metrics
summary = {
    "Strategy": [],
    "Sanity_Test_Success": [],
    "Exploit_Covered": []
}

for col in result_columns:
    sanity_success = sum(1 for x in df[col] if x.get('Sanity_Test_Success') is True)
    exploit_covered = sum(1 for x in df[col] if x.get('Exploit_Covered') is True)
    summary["Strategy"].append(col)
    summary["Sanity_Test_Success"].append(sanity_success)
    summary["Exploit_Covered"].append(exploit_covered)

summary_df = pd.DataFrame(summary)

summary_df

,Strategy,Sanity_Test_Success,Exploit_Covered
0,VL,84,49
1,pre_post_,62,72
2,pre,87,53
3,post,64,56
4,pre_VL_post,60,72
5,pre_VL,66,72
6,VL_post,57,74


In [3]:
# Add a new column for patches that passed both criteria
accepted_patch_counts = []

for col in result_columns:
    count = sum(
        1 for x in df[col]
        if x.get('Sanity_Test_Success') is True and x.get('Exploit_Covered') is True
    )
    accepted_patch_counts.append(count)

# Add the new column to the summary DataFrame
summary_df['Accepted_Patch'] = accepted_patch_counts

summary_df

,Strategy,Sanity_Test_Success,Exploit_Covered,Accepted_Patch
0,VL,84,49,28
1,pre_post_,62,72,33
2,pre,87,53,34
3,post,64,56,20
4,pre_VL_post,60,72,31
5,pre_VL,66,72,32
6,VL_post,57,74,30


In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast

# Load the CSV file with semicolon delimiter
file_path = 'validation_results_llama_4X.csv'  # adjust if necessary
df = pd.read_csv(file_path, delimiter=';')

# Define the strategy columns
result_columns = ['VL', 'pre_post_', 'pre', 'post', 'pre_VL_post', 'pre_VL', 'VL_post']

# Safely parse stringified dictionaries
def parse_result_column(value):
    try:
        return ast.literal_eval(value)
    except Exception:
        return {}

for col in result_columns:
    df[col] = df[col].apply(parse_result_column)

# Extract metrics into a long-form list of records
records = []
for _, row in df.iterrows():
    vuln_type = row["vulnerability_type"]
    for strategy in result_columns:
        data = row[strategy]
        if isinstance(data, dict):
            records.append({
                "Vulnerability_Type": vuln_type,
                "Strategy": strategy,
                "Sanity_Test_Success": int(data.get("Sanity_Test_Success", False)),
                "Exploit_Covered": int(data.get("Exploit_Covered", False)),
                "Accepted_Patch": int(data.get("Sanity_Test_Success", False) and data.get("Exploit_Covered", False))
            })

# Convert to DataFrame and aggregate
long_df = pd.DataFrame(records)
grouped = long_df.groupby(["Vulnerability_Type", "Strategy"]).sum().reset_index()



In [ ]:
# Re-define the plotting functions in this session

from math import pi

# Radar 1: Aggregated by Vulnerability_Type
radar1_grouped = grouped.groupby("Vulnerability_Type")[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch"]].sum().reset_index()

# Radar 2: Aggregated by Strategy
radar2_grouped = grouped.groupby("Strategy")[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch"]].sum().reset_index()


best_strategy_per_vuln = (
    grouped.loc[grouped.groupby("Vulnerability_Type")["Accepted_Patch"].idxmax()]
    [["Vulnerability_Type", "Strategy", "Accepted_Patch"]]
)

# Merge with radar1_grouped to align with plotting order
radar1_enriched = radar1_grouped.merge(best_strategy_per_vuln, on="Vulnerability_Type")


# Clean radar chart for multiple metrics with values annotated
def plot_radar_multi_clean(ax, labels, data, title, colors):
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    for i, metric in enumerate(data.columns[1:]):  # skip label column
        values = data[metric].tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=metric, color=colors[i])
        ax.fill(angles, values, alpha=0.1, color=colors[i])
        for j, val in enumerate(values[:-1]):
            ax.text(angles[j], val + 2, str(val), ha='center', va='bottom', fontsize=10, color='black', fontweight='bold')

    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=10)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_title(title, y=1.1, fontsize=14)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
    ax.spines["polar"].set_visible(False)

# Radar chart with per-axis offset annotations for best strategy
def plot_radar_with_best_strategy_offset(ax, labels, data, title, colors):
    N = len(labels)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    for i, metric in enumerate(data.columns[1:4]):  # Sanity, Exploit, Accepted
        values = data[metric].tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=metric, color=colors[i])
        ax.fill(angles, values, alpha=0.1, color=colors[i])
        for j, val in enumerate(values[:-1]):
            ax.text(angles[j], val + 2, str(val), ha='center', va='bottom', fontsize=10, color='black', fontweight='normal')

    # Custom offsets for each axis
    max_radius = max(data[["Sanity_Test_Success", "Exploit_Covered", "Accepted_Patch_x"]].max())
    custom_offsets = [23, 24, 43, 32, 23, 23, 31, -18, 28]  # Adjusted per axis

    for i, label in enumerate(labels):
        strategy = data.iloc[i]["Strategy"]
        count = data.iloc[i]["Accepted_Patch_y"]
        angle = angles[i]
        offset = custom_offsets[i] + max_radius
        ax.text(angle, offset, f'{strategy}\n({count})', ha='center', va='center', fontsize=9, color='black', fontweight='bold')

    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=10)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_title(title, y=1.1, fontsize=14)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
    ax.spines["polar"].set_visible(False)

# Now generate and save the two figures
import matplotlib.pyplot as plt

fig1, ax1 = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 7))
plot_radar_with_best_strategy_offset(
    ax1,
    radar1_enriched["Vulnerability_Type"],
    radar1_enriched,
    "Best Strategy per Vulnerability Type",
    ['#a6d854', '#66c2a5', '#8da0cb']
)
fig1.tight_layout()
fig1_path = "figures/radar_best_strategy_per_vuln.pdf"
fig1.savefig(fig1_path, format="pdf")
plt.close(fig1)

fig2, ax2 = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 7))
plot_radar_multi_clean(
    ax2,
    radar2_grouped["Strategy"],
    radar2_grouped,
    "Patch Performance by Strategy",
    ['#a6d854', '#66c2a5', '#8da0cb']
)
fig2.tight_layout()
fig2_path = "figures/radar_patch_performance_by_strategy.pdf"
fig2.savefig(fig2_path, format="pdf")
plt.close(fig2)

fig1_path, fig2_path



/var/folders/g0/fynz2n5945z5yjxd29pd5_v00000gn/T/ipykernel_46713/1135793230.py:111: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig1.show()
/var/folders/g0/fynz2n5945z5yjxd29pd5_v00000gn/T/ipykernel_46713/1135793230.py:112: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig2.show()
